In [1]:
import csv
from collections import defaultdict
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import pandas as pd

In [2]:
Fams,species={},[]
for r in csv.reader(open('Orthogroups_JSD.tsv'),delimiter='\t'):
    if r[0]=="Orthogroup":
        species=r[1:]
        print(species)
        continue
    fid,glt=r[0],r[1:]
    #print(fid,'>>>',glt)
    gbs=defaultdict(list)
    gbs={species[i]:gs.split(',') for i,gs in enumerate(glt)}
    Fams[fid]=gbs
    #break
        #if i==5: break
#species=list(set(species))


['Aque', 'Blan', 'Cele', 'Cgig', 'Chem', 'Ctel', 'Dgyr', 'Dmel', 'Drer', 'Eand', 'Hmia', 'Hrob', 'Hsap', 'Lana', 'Lgig', 'Lluy', 'Locu', 'Myes', 'Ngen', 'Nvec', 'Ofus', 'Paus', 'Skow', 'Smar', 'Smed', 'Spur', 'Tcas']


In [3]:
print(gbs)

{'Aque': [''], 'Blan': [''], 'Cele': [''], 'Cgig': [''], 'Chem': [''], 'Ctel': [''], 'Dgyr': [''], 'Dmel': [''], 'Drer': [''], 'Eand': [''], 'Hmia': [''], 'Hrob': [''], 'Hsap': [''], 'Lana': [''], 'Lgig': [''], 'Lluy': [''], 'Locu': [''], 'Myes': [''], 'Ngen': [''], 'Nvec': [''], 'Ofus': [''], 'Paus': [''], 'Skow': [''], 'Smar': [''], 'Smed': [''], 'Spur': [''], 'Tcas': ['Tcas_TC034497_001', ' Tcas_TC035044_001']}


In [4]:
genefam=defaultdict(dict)
for fid,gbs in Fams.items():
    for sp in gbs:
        for g in gbs[sp]:
            #genefam[sp][g.split('.',1)[0]]=fid
            genefam[sp][g]=fid
    

In [5]:
print(genefam.keys())

dict_keys(['Aque', 'Blan', 'Cele', 'Cgig', 'Chem', 'Ctel', 'Dgyr', 'Dmel', 'Drer', 'Eand', 'Hmia', 'Hrob', 'Hsap', 'Lana', 'Lgig', 'Lluy', 'Locu', 'Myes', 'Ngen', 'Nvec', 'Ofus', 'Paus', 'Skow', 'Smar', 'Smed', 'Spur', 'Tcas'])


In [6]:
print(list(genefam.items())[18:23])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
def gbm(mods):
    gbms=defaultdict(list)
    for g,m in mods.items():
        gbms[m].append(g)
    return(gbms)

In [8]:
ctel_mods={r[0]:r[1] for r in csv.reader(open('03-Capitella_teleta_mfuzz_clusters.txt'),delimiter='\t')}

In [9]:
ofus_mods={r[0]:r[1] for r in csv.reader(open('03-Owenia_fusiformis_mfuzz_clusters.txt'),delimiter='\t')}

In [10]:
ctel_mods.pop('Capitella_post1')

'8'

In [11]:
ctel_gbm=gbm(ctel_mods)
ofus_gbm=gbm(ofus_mods)


In [12]:
def comp(gl1,gl2,pairs):
    p1={r[0]:r[1] for r in pairs}
    p2={r[1]:r[0] for r in pairs}
    rgl1=[g1 for g1 in gl1 if g1 in p1]
    rgl2=[g2 for g2 in gl2 if g2 in p2]


In [13]:
print(sum([1 for f,gbs in Fams.items() if 'Ctel' in gbs and 'Ofus' in gbs]))


54440


In [14]:
print(list(ctel_gbm.items())[0:4])

[('10', ['CTELG00000000001.1', 'CTELG00000000002.2', 'CTELG00000000018.1', 'CTELG00000000030.1', 'CTELG00000000043.1', 'CTELG00000000044.2', 'CTELG00000000054.1', 'CTELG00000000064.1', 'CTELG00000000072.1', 'CTELG00000000113.1', 'CTELG00000000118.1', 'CTELG00000000121.1', 'CTELG00000000126.1', 'CTELG00000000128.1', 'CTELG00000000130.1', 'CTELG00000000137.1', 'CTELG00000000139.1', 'CTELG00000000141.1', 'CTELG00000000145.1', 'CTELG00000000149.1', 'CTELG00000000150.1', 'CTELG00000000164.1', 'CTELG00000000166.1', 'CTELG00000000171.1', 'CTELG00000000174.1', 'CTELG00000000175.1', 'CTELG00000000191.1', 'CTELG00000000197.1', 'CTELG00000000200.1', 'CTELG00000000201.1', 'CTELG00000000212.1', 'CTELG00000000214.1', 'CTELG00000000216.3', 'CTELG00000000224.1', 'CTELG00000000225.1', 'CTELG00000000226.1', 'CTELG00000000233.1', 'CTELG00000000239.1', 'CTELG00000000247.1', 'CTELG00000000249.1', 'CTELG00000000255.3', 'CTELG00000000272.1', 'CTELG00000000289.1', 'CTELG00000000290.2', 'CTELG00000000304.1', '

In [15]:
def ogcomp(gbm1,gbm2,genefam,nm1,nm2):
    res=[]
    nshog=sum([1 for f,gbs in Fams.items() if nm1 in gbs and nm2 in gbs])
    for m1,gl1 in gbm1.items():
        for m2,gl2 in gbm2.items():
            #print('pliv{} <-> spur{}'.format(m1,m2))
            rgl1=set([genefam[nm1][g1] for g1 in gl1 if g1 in genefam[nm1]])
            rgl2=set([genefam[nm2][g2] for g2 in gl2 if g2 in genefam[nm2]])
            gle=rgl1.union(rgl2)
            gli=rgl1.intersection(rgl2)
            # M : 
            hpd=stats.hypergeom(nshog, len(rgl1),len(rgl2))
            print(nshog,len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)))
            rep=round(len(gli)/(len(rgl2)*len(rgl1)/nshog),3)
            # (n * D) / N
            #print(len(p1),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep)
            res.append(['{}{}'.format(nm1,m1),'{}{}'.format(nm2,m2),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep])
            phyper=pd.DataFrame(res, columns = ['cl1','cl2','no1','no2','nov','pv','rf'])  
    return(phyper)

In [16]:
def comp(gbm1,gbm2,pairs,nm1,nm2):
    res=[]
    for m1,gl1 in gbm1.items():
        for m2,gl2 in gbm2.items():
            #print('pliv{} <-> spur{}'.format(m1,m2))
            p1={r[0]:r[1] for r in pairs}
            p2={r[1]:r[0] for r in pairs}
            rgl1=[g1 for g1 in gl1 if g1 in p1]
            rgl2=[g2 for g2 in gl2 if g2 in p2]
            gle=set([p2[g2] for g2 in gl2 if g2 in p2]+rgl1)
            gli=[g1 for g1 in rgl1 for g2 in rgl2 if p1[g1]==g2]
            hpd=stats.hypergeom(len(p1), len(rgl1),len(rgl2))
            rep=round(len(gli)/(len(rgl2)*len(rgl1)/len(p1)),3)# (n * D) / N
            #print(len(p1),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep)
            res.append(['{}{}'.format(nm1,m1),'{}{}'.format(nm2,m2),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep])
    phyper=pd.DataFrame(res, columns = ['cl1','cl2','no1','no2','nov','pv','rf'])  
    return(phyper)

In [17]:
ctel_ofus_hyp=ogcomp(ctel_gbm,ofus_gbm,genefam,'Ctel','Ofus')

54440 1496 881 86 3.806056364697122e-24
54440 1496 673 45 3.846938093092174e-08
54440 1496 1122 131 1.4556990607964953e-44
54440 1496 1227 152 4.9613658490086726e-55
54440 1496 1875 116 3.666898855111951e-16
54440 1496 889 111 1.3807233807766025e-40
54440 1496 722 55 1.2385677128657941e-11
54440 1496 874 47 6.974861357367425e-06
54440 1496 1390 53 0.0037589983236370624
54440 1496 630 41 2.906399744475016e-07
54440 1496 647 67 1.8441086143269463e-20
54440 1496 584 17 0.09548641974477289
54440 879 881 92 5.023209549979719e-46
54440 879 673 46 3.139164548486061e-16
54440 879 1122 32 0.0007882696070530907
54440 879 1227 47 4.2933018275019134e-08
54440 879 1875 95 9.096698892907763e-23
54440 879 889 41 2.172249519354121e-09
54440 879 722 51 1.950961453225728e-18
54440 879 874 43 1.3045421697596714e-10
54440 879 1390 52 1.806342887946316e-08
54440 879 630 32 1.3415758478352726e-08
54440 879 647 53 5.889108818339535e-22
54440 879 584 17 0.00783966046497598
54440 564 881 16 0.01149559486570904

In [18]:
print(ctel_ofus_hyp)

        cl1     cl2   no1   no2  nov            pv     rf
0    Ctel10   Ofus8  1496   881   86  3.806056e-24  3.552
1    Ctel10   Ofus9  1496   673   45  3.846938e-08  2.433
2    Ctel10  Ofus11  1496  1122  131  1.455699e-44  4.249
3    Ctel10  Ofus10  1496  1227  152  4.961366e-55  4.508
4    Ctel10   Ofus2  1496  1875  116  3.666899e-16  2.251
..      ...     ...   ...   ...  ...           ...    ...
139   Ctel9   Ofus5   605   874   36  2.208059e-11  3.706
140   Ctel9   Ofus1   605  1390   40  4.279238e-08  2.589
141   Ctel9   Ofus6   605   630   43  4.034915e-21  6.142
142   Ctel9   Ofus7   605   647   45  2.257892e-22  6.259
143   Ctel9   Ofus4   605   584    7  1.477960e-01  1.079

[144 rows x 7 columns]


In [19]:
ctel_ofus_hyp.to_csv('ctel_ofus_phyper_newOfusClust_ALL.tsv',sep='\t')